# Get a raw image, display it and superimpose the position of the reference sources

In [3]:
#dataid = {'visit':2249, 'raft':'R32', 'ccd':'S21'}
#dataid = {'visit':277108, 'raftName':'R34', 'detectorName':'S22'}
#dataid = {'visit':277108, 'raftName':'R43', 'detectorName':'S22'}
#dataid = {'visit':230, 'raftName':'R43', 'detectorName':'S22'}
dataid = {'visit':179972, 'raftName':'R41', 'detectorName':'S20'}
#dataid = {'visit': 263504, 'filter': 'i', 'raftName': 'R33', 'detectorName': 'S01'}
#dataid = {'visit': 227032, 'filter': 'z', 'raftName': 'R32', 'detectorName': 'S20'}

In [4]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
import numpy

#butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_30")
butler = dafPersist.Butler("/sps/lsst/users/lsstprod/desc/DC2-test/phosim/input/rerun/test")

raw = butler.get('raw', dataid)
wcsRaw = raw.getWcs()

In [75]:
# Display image in ds9 (the display_ds9 stack package should be set up)
ds9.mtv(raw.getMaskedImage().getImage(),frame=1,wcs=wcsRaw)

## Create and load a catalog with reference objects 

In [5]:
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
from lsst.meas.algorithms import LoadIndexedReferenceObjectsTask

# Configure LoadIndexedReferenceObjectsTask
refConfig = LoadIndexedReferenceObjectsTask.ConfigClass()
refConfig.ref_dataset_name = 'cal_ref_cat'
refTask = LoadIndexedReferenceObjectsTask(butler, config=refConfig)

#define a reference filter (not critical for what we are doing) 
filter = 'lsst_r_smeared'

# Find the approximate celestial coordinates of the sensor's center 
centerPixel = afwGeom.Point2D(2000., 2000.)
centerCoord = wcsRaw.pixelToSky(centerPixel)

# Retrieve reference object within a 0.5 deg radius circle around the sensor's center
radius = afwGeom.Angle(0.5, afwGeom.degrees)
ref = refTask.loadSkyCircle(centerCoord, radius, filter).refCat.copy(deep=True)

In [6]:
# We can now display the reference catalog as an astropy table
ref.asAstropy()

id,coord_ra,coord_dec,parent,lsst_u_smeared_flux,lsst_u_smeared_instFlux,lsst_g_smeared_flux,lsst_g_smeared_instFlux,lsst_r_smeared_flux,lsst_r_smeared_instFlux,lsst_i_smeared_flux,lsst_i_smeared_instFlux,lsst_z_smeared_flux,lsst_z_smeared_instFlux,lsst_y_smeared_flux,lsst_y_smeared_instFlux,lsst_u_smeared_fluxSigma,lsst_u_smeared_instFluxSigma,lsst_u_smeared_instFluxErr,lsst_g_smeared_fluxSigma,lsst_g_smeared_instFluxSigma,lsst_g_smeared_instFluxErr,lsst_r_smeared_fluxSigma,lsst_r_smeared_instFluxSigma,lsst_r_smeared_instFluxErr,lsst_i_smeared_fluxSigma,lsst_i_smeared_instFluxSigma,lsst_i_smeared_instFluxErr,lsst_z_smeared_fluxSigma,lsst_z_smeared_instFluxSigma,lsst_z_smeared_instFluxErr,lsst_y_smeared_fluxSigma,lsst_y_smeared_instFluxSigma,lsst_y_smeared_instFluxErr,resolved,variable,raJ2000,decJ2000,sigma_raJ2000,sigma_decJ2000,lsst_u,lsst_g,lsst_r,lsst_i,lsst_z,lsst_y,properMotionRa,properMotionDec,parallax,radialVelocity,centroid_x,centroid_y,hasCentroid
,rad,rad,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool
1277260946436,0.9969151355857375,-0.49710819972859266,-1,1.6612178496166186e-07,1.6612178496166186e-07,1.5200021313468859e-06,1.5200021313468859e-06,4.544919432474331e-06,4.544919432474331e-06,1.2247206736161752e-05,1.2247206736161752e-05,1.9004741594090546e-05,1.9004741594090546e-05,2.3364314790439987e-05,2.3364314790439987e-05,1.5300381826971403e-10,1.5300381826971403e-10,1.5300381826971403e-10,1.3999736995834068e-09,1.3999736995834068e-09,1.3999736995834068e-09,4.1860254936297416e-09,4.1860254936297416e-09,4.1860254936297416e-09,1.1280094264600926e-08,1.1280094264600926e-08,1.1280094264600926e-08,1.750401387630272e-08,1.750401387630272e-08,1.750401387630272e-08,2.151932917779497e-08,2.151932917779497e-08,2.151932917779497e-08,False,False,57.1190298,-28.4822018,2.777777778e-08,2.777777778e-08,25.85026453,23.44591757,22.25519938,21.17972813,20.70401292,20.47906653,0.0,0.0,0.0,0.0,nan,nan,False
992912206852,0.9967867872240551,-0.49712065591343446,-1,2.0079813782963007e-06,2.0079813782963007e-06,1.8423759038077915e-05,1.8423759038077915e-05,5.847545057755917e-05,5.847545057755917e-05,0.00011657152793176828,0.00011657152793176828,0.00016030258617245693,0.00016030258617245693,0.00018794306491552382,0.00018794306491552382,1.84941919546988e-09,1.84941919546988e-09,1.84941919546988e-09,1.696890916719701e-08,1.696890916719701e-08,1.696890916719701e-08,5.385788032239913e-08,5.385788032239913e-08,5.385788032239913e-08,1.0736634499329149e-07,1.0736634499329149e-07,1.0736634499329149e-07,1.4764413811563712e-07,1.4764413811563712e-07,1.4764413811563712e-07,1.7310195984243895e-07,1.7310195984243895e-07,1.7310195984243895e-07,False,False,57.111676,-28.482915499999997,2.777777778e-08,2.777777778e-08,23.14431291,20.73602046,19.48109115,18.73324644,18.3887578,18.21456429,0.0,0.0,0.0,0.0,nan,nan,False
1277260281860,0.9967296833142314,-0.4969892380869451,-1,8.864889303663751e-07,8.864889303663751e-07,8.133287961764624e-06,8.133287961764624e-06,2.58492015557984e-05,2.58492015557984e-05,5.1574721193864974e-05,5.1574721193864974e-05,7.068004493082869e-05,7.068004493082869e-05,8.283015249357009e-05,8.283015249357009e-05,8.164864784663408e-10,8.164864784663408e-10,8.164864784663408e-10,7.49103504711486e-09,7.49103504711486e-09,7.49103504711486e-09,2.3807994467271958e-08,2.3807994467271958e-08,2.3807994467271958e-08,4.750207367852703e-08,4.750207367852703e-08,4.750207367852703e-08,6.509872713195022e-08,6.509872713195022e-08,6.509872713195022e-08,7.628938975284724e-08,7.628938975284724e-08,7.628938975284724e-08,False,False,57.1084041999

## Display the bright reference sources as a small circle overlaid on the ds9 image

In [78]:
for r in ref:
    ra = r.get("coord_ra")
    dec = r.get("coord_dec")
    mag = r.get("lsst_z")
    c = afwGeom.SpherePoint(ra, dec)
    pt = wcsRaw.skyToPixel(c)
    # Ignore sources which lye outside the sensor footprint or which are too faint 
    if (pt.getX() > 4000) or (pt.getX()<0) or (pt.getY()>4000) or (pt.getY()<0) or mag > 21:
        continue
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=1, size=8)

### Note that reference sources does not exactly match the sources in the raw images according to Scott 
### this is due to the fact that PhoSim  does not include an internal model for the motion of the earth

## We can now redo the same thing with the calibrated exposure (calexp)

In [79]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
import numpy

# We need to get a new butler as calexp are in the output directory while eImage(raw) was in the input

#butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_18/data/output")
#butler = dafPersist.Butler("/sps/lsst/users/lsstprod/desc/DC2-test/newCam/input/rerun/jointcal2")
#butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_30/rerun/210918")
calexp = butler.get('calexp', dataid)
wcs = calexp.getWcs()

In [80]:
# display in another frame
ds9.mtv(calexp.getMaskedImage().getImage(),frame=2,wcs=wcs)

In [ ]:
# display in another frame
ds9.mtv(calexp.getMaskedImage(),frame=2,wcs=wcs)

In [ ]:
import lsst.afw.geom as afwGeom
meta = butler.get('calexp_md', dataid)
wcs = afwGeom.makeSkyWcs(meta)
bbox = butler.get('calexp_bbox', dataid)
butler.getKeys()

In [81]:
for r in ref:
    ra = r.get("coord_ra")
    dec = r.get("coord_dec")
    mag = r.get("lsst_u")
    c = afwGeom.SpherePoint(ra, dec)
    pt = wcs.skyToPixel(c)
    # Ignore sources which lie outside the sensor footprint or which are too faint 
    if (pt.getX() > 4000) or (pt.getX()<0) or (pt.getY()>4000) or (pt.getY()<0) or mag > 23:
        continue
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=2, size=8)

In [ ]:
import lsst.afw.geom as afwGeom
src = butler.get('src', dataid)
for s in src:
    ra = s.get("coord_ra")
    dec = s.get("coord_dec")
    c = afwGeom.SpherePoint(ra, dec)
    pt = wcs.skyToPixel(c)
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=2, size=8)

In [ ]:
ra = []
dec = []
for r in ref:
    mag = r.get("lsst_z")
#    c = afwCoord.IcrsCoord(ra, dec)
#    pt = wcsRaw.skyToPixel(c)
    if mag > 21:
        continue
    ra.append(afwGeom.radToDeg(r.get("coord_ra")))
    dec.append(afwGeom.radToDeg(r.get("coord_dec")))

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pylab as plt
import numpy as np

fig, (ax0) = plt.subplots(ncols=1, figsize=(10, 10))
_ = ax0.scatter(ra, dec, s=0.5)

In [ ]:
ref['id']%1024

In [ ]:
srcA = butler.get('src', dataid).asAstropy()
refA = ref.asAstropy()

In [ ]:
fluxType = "base_CircularApertureFlux_6_0"

Flags = ["base_PixelFlags_flag_saturated", "base_PixelFlags_flag_cr", "base_PixelFlags_flag_interpolated",
         fluxType + "_flag", "base_SdssCentroid_flag", 
         "base_SdssCentroid_flag_almostNoSecondDerivative", "base_SdssCentroid_flag_edge", "base_SdssCentroid_flag_noSecondDerivative",
         "base_SdssCentroid_flag_notAtMaximum", "base_SdssCentroid_flag_resetToPeak", 
         "base_SdssShape_flag", "base_ClassificationExtendedness_flag"]

# select sources
cut = np.ones_like(srcA['id'], dtype=bool)
for flag in Flags:
    cut &= srcA[flag]==False
cut &= (srcA[fluxType + '_flux'] > 0) & (srcA[fluxType + '_flux'] / srcA[fluxType + '_fluxSigma'] > 5)
#cut &= srcA['base_ClassificationExtendedness_value'] < 0.5

cat = srcA[cut]['id', 'coord_ra', 'coord_dec']

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

cut1 = (refA['id']%1024 == 231)
cut2 = (refA['id']%1024 == 4)

cRef = SkyCoord(ra = refA['coord_ra'], dec = refA['coord_dec'])
cRef1 = SkyCoord(ra = refA[cut1]['coord_ra'], dec = refA[cut1]['coord_dec'])
cRef2 = SkyCoord(ra = refA[cut2]['coord_ra'], dec = refA[cut2]['coord_dec'])
cCat = SkyCoord(ra = cat['coord_ra'], dec = cat['coord_dec'])

In [ ]:
idx, d2d, d3d = cCat.match_to_catalog_sky(cRef)
idx1, d2d1, d3d1 = cCat.match_to_catalog_sky(cRef1)
idx2, d2d2, d3d2 = cCat.match_to_catalog_sky(cRef2)

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(15, 8))
_ = ax0.hist(d2d.arcsec, bins=50, range=[0, 20])
_ = ax1.hist(d2d1.arcsec, bins=50, range=[0,20])
_ = ax2.hist(d2d2.arcsec, bins=50, range=[0,20])
ax0.set_xlabel('Ang dist in arcsec star + galaxies')
ax1.set_xlabel('Ang dist in arcsec galaxies only')
ax2.set_xlabel('Ang dist in arcsec star only')
fig.tight_layout()

In [ ]:
fig.savefig("refCat.png")

## Same thing with coadd

In [1]:
import lsst.daf.persistence as dafPersist
import lsst.afw.display.ds9 as ds9
import numpy

dataid = {'tract': 4849, 'patch': '5,4', 'filter': 'u' }
butler = dafPersist.Butler("/sps/lsst/dataproducts/desc/DC2/Run1.2p/w_2018_30/rerun/coadd-all2")

coadd = butler.get('deepCoadd', dataid)
wcs = coadd.getWcs()

In [3]:
ds9.mtv(coadd.getMaskedImage().getImage(),frame=1,wcs=wcs)

In [ ]:
import lsst.afw.geom as afwGeom
src = butler.get('deepCoadd_meas', dataid)
for s in src:
    ra = s.get("coord_ra")
    dec = s.get("coord_dec")
    c = afwGeom.SpherePoint(ra, dec)
    pt = wcs.skyToPixel(c)
    with ds9.Buffering():
        ds9.dot('o', pt.getX(), pt.getY(), frame=1, size=8)